# About
Convert ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19 to Core ML format


- Failed to inspect the frozen_graph because TFLite_Detection_PostProcess is not supported by normal TensorFlow verison.


- Failed to convert to Core ML because coremltools can not open frozen graph  if TF2 is used.

# Download model and setup enviroments

In [1]:
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz
!tar -zxvf ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz

--2021-08-11 20:04:35--  http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.213.128, 2607:f8b0:400c:c0a::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.213.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156413934 (149M) [application/x-tar]
Saving to: ‘ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz’

ssdlite_mobiledet_e 100%[===================>] 149.17M   144MB/s    in 1.0s    

2021-08-11 20:04:37 (144 MB/s) - ‘ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz’ saved [156413934/156413934]

ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/model.ckpt-400000.data-00000-of-00001
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/model.ckpt-400000.index
ssdlite_mobiledet_

In [2]:
!pip install coremltools==4.1

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=7a59869465114514fabb0581872b84427e3872ad81ff1534909cbb92e950cc8d
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [3]:
%tensorflow_version 2.x

import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import coremltools as ct
print(ct.__version__)

3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
2.5.0


4.1


# Inspect the graph

In [5]:
def load_frozen_graph(path):
  with tf.io.gfile.GFile(path, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    return graph_def


def export_ops_name(the_graph, filename):
  with open(filename, "w") as text_file:
    ops = the_graph.get_operations()
    N = len(ops)
    for i in range(N):
      text_file.write('\n\nop id {} , op type: "{}"'.format(str(i), ops[i].type))
      
      text_file.write('\ninput(s):'),
      for x in ops[i].inputs:
        text_file.write("name = {}, shape: {}, ".format(x.name, x.get_shape()))
      
      text_file.write('\noutput(s):'),
      for x in ops[i].outputs:
        text_file.write("name = {}, shape: {},".format(x.name, x.get_shape()))
  print('Exported to:', filename)


model_name = 'ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19'
input_file = f'{model_name}/fp32/tflite_graph.pb'

with tf.compat.v1.Session(graph=tf.Graph()) as sess:
  graph_def = load_frozen_graph(input_file)
  tf.import_graph_def(graph_def, name="")
  the_graph = tf.compat.v1.get_default_graph()
  export_ops_name(the_graph, f'{model_name}_ops.txt')


NotFoundError: ignored

# Convert to Core ML

In [6]:
input_width = 320
input_height = 320

tfmodel = '/content/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/tflite_graph.pb'

image_input = ct.ImageType(shape=(1, input_width, input_height, 3),
                           bias=[-1,-1,-1], scale=2./255)

coreml_model = ct.convert(
    tfmodel,
    source='tensorflow',
    inputs=[image_input]
)

print(coreml_model)

NotImplementedError: ignored